In [105]:
import copy

In [319]:
class LiquidityPool():

    def __init__(self, year_interest_rate = 36.5):
        """ Creates liquidity pool """

        self.year_interest_rate = year_interest_rate

        # 3x variables to keep track of
        self.lenders = {}
        self.borrowers = {}
        self.liquidity_pool_lenders = 0
        self.liquidity_pool_borrows = 0
        self.interest_rate_factor = 1

        self.time_stamp = 0

        # 3x variable purely for debugging purposes
        self.interest_checker = 0
        self.USDC_add_checker = 0
        self.USDC_remove_checker = 0


    def deposit(self, user_name, USDC_amount, time_gap):
        """ User deposits USDC amount into liquidity pool with time_gap since last transaction """
        # Update time stamp, interest_rate factor, and liquidity pool
        self.time_stamp += time_gap
        self.interest_rate_factor_update(time_gap)
        self.liquidity_pool_update(time_gap, USDC_amount, 0)

        # Create new user if one does not exist
        if user_name not in self.lenders:
            self.lenders[user_name] = {"LP_tokens":0, "LP_factor":0}
        
        # Update a users token, and their LP_factor
        temp_start_balance = copy.deepcopy(self.lenders[user_name]) # ONLY FOR DEBUGGING
        self.lenders[user_name]["LP_factor"] = ((USDC_amount / self.interest_rate_factor) +
                                            self.lenders[user_name]["LP_factor"] * self.lenders[user_name]["LP_tokens"]) / (USDC_amount + self.lenders[user_name]["LP_tokens"]) 
        self.lenders[user_name]["LP_tokens"] += USDC_amount

        # ONLY FOR DEBUGGING
        print(f'{user_name}, Deposit (USDC): {USDC_amount}, (Prior: {temp_start_balance}, Now: {self.lenders[user_name]})') 
        self.USDC_add_checker += USDC_amount


    def withdraw(self, user_name, token_amount, time_gap):
        """ User withdraws a certain amount of LP tokens with time_gap since last transaction"""
        # Update time stamp, interest_rate factor
        self.time_stamp += time_gap
        self.interest_rate_factor_update(time_gap)

        # Calculate how much USDC to return and update lenders
        temp_start_balance = self.lenders[user_name] # ONLY FOR DEBUGGING
        USDC_amount = token_amount * self.lenders[user_name]["LP_factor"] * self.interest_rate_factor
        self.lenders[user_name]["LP_tokens"] -= token_amount

        # Update liquidity pool for amount taken out (principal + interest)
        self.liquidity_pool_update(time_gap, 0, USDC_amount)

        # Remove user if now empty
        if self.lenders[user_name]["LP_tokens"] == 0:
            self.lenders.pop(user_name)

        # ONLY FOR DEMBUGGING
        print(f'Withdraw (USDC): {USDC_amount}, (Prior: {temp_start_balance}, Now: {self.lenders.get(user_name,0)})')
        self.USDC_remove_checker += USDC_amount

    def borrow(self, user_name, USDC_amount, time_gap):
        """ User borrowers a certain amount of USDC at a specific time_gap since last transaction """
        # Update time stamp, interest rate factor, and liquidity pool (we only update for inflows / outflows from lenders)
        self.time_stamp += time_gap
        self.interest_rate_factor_update(time_gap)
        self.liquidity_pool_update(time_gap, 0, 0)

        # Create borrower loan in ledger, and update borrow amount for that specific borrower
        if user_name not in self.borrowers:
            self.borrowers[user_name] = {"borrow_amount":0, "loans":[]}
        self.borrowers[user_name]["loans"].append([USDC_amount, self.time_stamp])
        self.borrowers[user_name]["borrow_amount"] += USDC_amount
        self.liquidity_pool_borrows += USDC_amount

        # ONLY FOR DEBUGGING
        print(f'Borrow (USDC): {USDC_amount}, {user_name} Loans: {self.borrowers[user_name]})') 

    def repay(self, user_name, USDC_amount, loan_number, time_gap):
        """ User repays a certain amount of USDC at a specific time_gap since last transaction """
        # Update time stamp, interest rate factor, and liquidity pool (we only update for inflows / outflows from lenders)
        self.time_stamp += time_gap
        self.interest_rate_factor_update(time_gap)
        self.liquidity_pool_update(time_gap, 0, 0)

        # Calculate the interest earnt so far on that specific loan
        interest = self.interest_income_calculator(self.time_stamp - self.borrowers[user_name]["loans"][loan_number][1], self.borrowers[user_name]["loans"][loan_number][0])
        temp_loan_balance = copy.deepcopy(self.borrowers[user_name]["loans"]) # Only for debugging

        if USDC_amount == "all":
            # Reduce borrowing amount and borrower ledger by principal, calculate how much USDC they have to pay. Remove that loan.
            self.liquidity_pool_borrows -= self.borrowers[user_name]["loans"][loan_number][0]
            self.borrowers[user_name]["borrow_amount"] -= self.borrowers[user_name]["loans"][loan_number][0]
            print(f'{user_name} needs to pay {self.borrowers[user_name]["loans"][loan_number][0]} principal + interest {interest} to cancel loan {loan_number}')
            self.borrowers[user_name]["loans"].pop(loan_number)
        else:
            # Reduce liquidity pool by USDC amount. Update the amount outstanding on the loan.
            self.liquidity_pool_borrows -= USDC_amount
            self.borrowers[user_name]["borrow_amount"] -= USDC_amount
            self.borrowers[user_name]["loans"][loan_number][0] *= (1-(USDC_amount/(self.borrowers[user_name]["loans"][loan_number] + interest)))

        # ONLY FOR DEBUGGING
        print(f'{user_name}, Loans Before: {temp_loan_balance}, Loans After: {self.borrowers[user_name]})') 


    def interest_income_calculator(self, time_gap, amount):
        """ Calculate interest earn't on specific amount in a given time gap """
        return amount * (time_gap / 31536000) * (self.year_interest_rate / 100)


    def liquidity_pool_update(self, time_gap, inflow, outflow):
        """ Update the liquidity pool for interest, outflows and inflows """
        interest = self.interest_income_calculator(time_gap, self.liquidity_pool_borrows)
        self.interest_checker += interest
        self.liquidity_pool_lenders = self.liquidity_pool_lenders + inflow - outflow + interest


    def interest_rate_factor_update(self, time_gap):
        """ Update the interest rate factor since the last transaction """
        if self.liquidity_pool_borrows != 0:
            self.interest_rate_factor = self.interest_rate_factor * (1+ (time_gap / 31536000) * (self.year_interest_rate / 100) * (self.liquidity_pool_borrows / self.liquidity_pool_lenders))

# Only thing to work out is how to apportion the interest on borrowers side. Should we update all of them every time a transaction is made? makes the most sense?


In [320]:
lp = LiquidityPool()

# Journey  (1 borrower / 1 lender)
Matthew deposits 2 amounts to begin. Ben borrows 800 for 1 day. Matthew withdraws 10 days later. Matthew earns 0.8 interest as expected, Ben pays 800 principal and 0.8 interest.

In [321]:
# Matthew deposits 400 USDC to start
lp.deposit("Matthew", 400, 0)

Matthew, Deposit (USDC): 400, (Prior: {'LP_tokens': 0, 'LP_factor': 0}, Now: {'LP_tokens': 400, 'LP_factor': 1.0})


In [322]:
# Matthew deposits 400 USDC a day later
lp.deposit("Matthew", 400, 86400)

Matthew, Deposit (USDC): 400, (Prior: {'LP_tokens': 400, 'LP_factor': 1.0}, Now: {'LP_tokens': 800, 'LP_factor': 1.0})


In [323]:
# Ben Borrows 800 USDC a day later
lp.borrow("Ben", 800, 86400)

Borrow (USDC): 800, Ben Loans: {'borrow_amount': 800, 'loans': [[800, 172800]]})


In [324]:
# Ben repays loan a day later (clearly charged 0.1% interest)
lp.repay("Ben", "all", 0, 86400)

Ben needs to pay 800 principal + interest 0.7999999999999999 to cancel loan 0
Ben, Loans Before: [[800, 172800]], Loans After: {'borrow_amount': 0, 'loans': []})


In [325]:
# Matthew withdraws all tokens 10 days later
lp.withdraw("Matthew", 800, 86400*10)

Withdraw (USDC): 800.8, (Prior: {'LP_tokens': 0, 'LP_factor': 1.0}, Now: 0)


In [326]:
print(f'Lenders {lp.lenders}')
print(f'Borrowers {lp.borrowers}')
print(f'Liquidity Pool Lenders {lp.liquidity_pool_lenders}')
print(f'Liquidity Pool Borrowers {lp.liquidity_pool_borrows}')
print(f'Interest checker {lp.interest_checker}')
print(f'USDC add checker {lp.USDC_add_checker}')
print(f'USDC remove checker {lp.USDC_remove_checker}')

Lenders {}
Borrowers {'Ben': {'borrow_amount': 0, 'loans': []}}
Liquidity Pool Lenders 0.0
Liquidity Pool Borrowers 0
Interest checker 0.7999999999999999
USDC add checker 800
USDC remove checker 800.8


# Journey 2 (2 borrowers / 1 lenders)
Matthew and Khibar both deposit at different times. Khibar half of what Matthew does. Therefore interest should be less for Khibar. Notice how Matthew gets twice the interest Khibar does.

In [327]:
lp = LiquidityPool()

In [328]:
# Matthew deposits 400 USDC to start.
lp.deposit("Matthew", 400, 0)

Matthew, Deposit (USDC): 400, (Prior: {'LP_tokens': 0, 'LP_factor': 0}, Now: {'LP_tokens': 400, 'LP_factor': 1.0})


In [329]:
# Khibar deposits 200 USDC a day later.
lp.deposit("Khibar", 200, 86400)

Khibar, Deposit (USDC): 200, (Prior: {'LP_tokens': 0, 'LP_factor': 0}, Now: {'LP_tokens': 200, 'LP_factor': 1.0})


In [330]:
# Ben Borrows 600 USDC a day later
lp.borrow("Ben", 600, 86400)

Borrow (USDC): 600, Ben Loans: {'borrow_amount': 600, 'loans': [[600, 172800]]})


In [331]:
# Ben repays loan a day later (clearly charged 0.1% interest)
lp.repay("Ben", "all", 0, 86400)

Ben needs to pay 600 principal + interest 0.6 to cancel loan 0
Ben, Loans Before: [[600, 172800]], Loans After: {'borrow_amount': 0, 'loans': []})


In [332]:
# Matthew withdraws all tokens 10 days later
lp.withdraw("Matthew", 400, 86400*10)

Withdraw (USDC): 400.4, (Prior: {'LP_tokens': 0, 'LP_factor': 1.0}, Now: 0)


In [333]:
# Matthew withdraws all tokens 10 days later
lp.withdraw("Khibar", 200, 86400*10)

Withdraw (USDC): 200.2, (Prior: {'LP_tokens': 0, 'LP_factor': 1.0}, Now: 0)


In [334]:
print(f'Lenders {lp.lenders}')
print(f'Borrowers {lp.borrowers}')
print(f'Liquidity Pool Lenders {lp.liquidity_pool_lenders}')
print(f'Liquidity Pool Borrowers {lp.liquidity_pool_borrows}')
print(f'Interest checker {lp.interest_checker}')
print(f'USDC add checker {lp.USDC_add_checker}')
print(f'USDC remove checker {lp.USDC_remove_checker}')

Lenders {}
Borrowers {'Ben': {'borrow_amount': 0, 'loans': []}}
Liquidity Pool Lenders 5.684341886080802e-14
Liquidity Pool Borrowers 0
Interest checker 0.6
USDC add checker 600
USDC remove checker 600.5999999999999


# Journey 2 (2 borrowers / 2 lenders)
In this scenario Matthew deposits, a loan is taken out, and then Khibar deposits. As a result Matthew should earn a higher portion of interest. Matthew first deposits 500, and Ben takes out a loan of 500 a day later. A a day after this Khibar enters the pool with a deposit of 500. This means Matthew should have earnt a whole day of interest (0.5) before Khibar enters the pool. In the end Matthew gets 0.75 interest (0.5 + 0.25) and Khibar gets 0.25. All as expected.

In [335]:
lp = LiquidityPool()

In [336]:
# Matthew deposits 500 USDC to start.
lp.deposit("Matthew", 500, 0)

Matthew, Deposit (USDC): 500, (Prior: {'LP_tokens': 0, 'LP_factor': 0}, Now: {'LP_tokens': 500, 'LP_factor': 1.0})


In [337]:
# Ben borrows 500 a day later
lp.borrow("Ben", 500, 86400)

Borrow (USDC): 500, Ben Loans: {'borrow_amount': 500, 'loans': [[500, 86400]]})


In [338]:
# Khibar deposits 500 a day later
lp.deposit("Khibar", 500, 86400)

Khibar, Deposit (USDC): 500, (Prior: {'LP_tokens': 0, 'LP_factor': 0}, Now: {'LP_tokens': 500, 'LP_factor': 0.999000999000999})


In [339]:
# Ben repaysborrows 500 a day later (so had it 2 days in total)
lp.repay("Ben", "all", 0, 86400)

Ben needs to pay 500 principal + interest 0.9999999999999999 to cancel loan 0
Ben, Loans Before: [[500, 86400]], Loans After: {'borrow_amount': 0, 'loans': []})


In [340]:
# Matthew withdraws 10 days later
lp.withdraw("Matthew", 500, 86400*10)

Withdraw (USDC): 500.75012493753115, (Prior: {'LP_tokens': 0, 'LP_factor': 1.0}, Now: 0)


In [341]:
# Khibar withdraws 10 days later
lp.withdraw("Khibar", 500, 86400*10)

Withdraw (USDC): 500.2498750624687, (Prior: {'LP_tokens': 0, 'LP_factor': 0.999000999000999}, Now: 0)


In [342]:
print(f'Lenders {lp.lenders}')
print(f'Borrowers {lp.borrowers}')
print(f'Liquidity Pool Lenders {lp.liquidity_pool_lenders}')
print(f'Liquidity Pool Borrowers {lp.liquidity_pool_borrows}')
print(f'Interest checker {lp.interest_checker}')
print(f'USDC add checker {lp.USDC_add_checker}')
print(f'USDC remove checker {lp.USDC_remove_checker}')

Lenders {}
Borrowers {'Ben': {'borrow_amount': 0, 'loans': []}}
Liquidity Pool Lenders 1.7053025658242404e-13
Liquidity Pool Borrowers 0
Interest checker 0.9999999999999999
USDC add checker 1000
USDC remove checker 1000.9999999999998


# Journey 3 (1 borrower / 1 lender - multiple borrows)
Matthew deposits 500. Ben borrows 250 a day later. Then 250 another day later. He repays one loan a day after, and another loan a day after that. Given both loans have been borrowed for 2 days the interest should be the same.

In [343]:
lp = LiquidityPool()

In [344]:
# Matthew deposits 500 USDC to start.
lp.deposit("Matthew", 500, 0)

Matthew, Deposit (USDC): 500, (Prior: {'LP_tokens': 0, 'LP_factor': 0}, Now: {'LP_tokens': 500, 'LP_factor': 1.0})


In [345]:
# Ben borrows 500 a day later
lp.borrow("Ben", 250, 86400)

Borrow (USDC): 250, Ben Loans: {'borrow_amount': 250, 'loans': [[250, 86400]]})


In [346]:
# Ben borrows 500 a day later
lp.borrow("Ben", 250, 86400)

Borrow (USDC): 250, Ben Loans: {'borrow_amount': 500, 'loans': [[250, 86400], [250, 172800]]})


In [347]:
# Ben repaysborrows 500 a day later (so had it 2 days in total)
lp.repay("Ben", "all", 0, 86400)

Ben needs to pay 250 principal + interest 0.49999999999999994 to cancel loan 0
Ben, Loans Before: [[250, 86400], [250, 172800]], Loans After: {'borrow_amount': 250, 'loans': [[250, 172800]]})


In [348]:
# Ben repaysborrows 500 a day later (so had it 2 days in total)
lp.repay("Ben", "all", 0, 86400)

Ben needs to pay 250 principal + interest 0.49999999999999994 to cancel loan 0
Ben, Loans Before: [[250, 172800]], Loans After: {'borrow_amount': 0, 'loans': []})


In [349]:
# Matthew deposits 500 USDC to start.
lp.withdraw("Matthew", 500, 0)

Withdraw (USDC): 500.9999999999999, (Prior: {'LP_tokens': 0, 'LP_factor': 1.0}, Now: 0)


In [318]:
print(f'Lenders {lp.lenders}')
print(f'Borrowers {lp.borrowers}')
print(f'Liquidity Pool Lenders {lp.liquidity_pool_lenders}')
print(f'Liquidity Pool Borrowers {lp.liquidity_pool_borrows}')
print(f'Interest checker {lp.interest_checker}')
print(f'USDC add checker {lp.USDC_add_checker}')
print(f'USDC remove checker {lp.USDC_remove_checker}')

Lenders {}
Borrowers {'Ben': {'borrow_amount': 0, 'loans': []}}
Liquidity Pool Lenders 1.1368683772161603e-13
Liquidity Pool Borrowers 0
Interest checker 0.9999999999999999
USDC add checker 500
USDC remove checker 500.9999999999999
